### This script is used to run tests at the bug report commit (aka the commit just before the bug report is submitted) and collect the coverage

In [9]:
# data
import pandas as pd

df_pattern = pd.read_csv("/Users/IceCream/Documents/icse23_bias/data/pattern.csv")
df_p4 = df_pattern[df_pattern['pattern']==4]
display(df_p4)

,project,bugid,test,test_method,pattern,datetime_modified,commit_modified,created,report_creation,report_resolution,modified_in,created_in,report_url
2,Cli,3,PatternOptionBuilderTest,testSimplePattern,4,"[Timestamp('2002-06-10 14:01:16'), Timestamp('...","['aae50c585ec3ac33c6a9af792e80378904a73195', '...","[True, False, False, False, False, False, False]",2002-10-24 22:32:22,2007-03-15 16:49:08,False,False,https://issues.apache.org/jira/browse/CLI-1
3,Cli,4,OptionsTest,testMissingOptionException,4,[Timestamp('2007-06-05 00:01:08')],['f78959477b207bf710049aa35730ef7659f0a1b9'],[True],2002-10-24 22:32:22,2007-03-15 16:49:08,False,False,https://issues.apache.org/jira/browse/CLI-1
4,Cli,4,OptionsTest,testMissingOptionsException,4,[Timestamp('2007-06-05 00:01:08')],['f78959477b207bf710049aa35730ef7659f0a1b9'],[True],2002-10-24 22:32:22,2007-03-15 16:49:08,False,False,https://issues.apache.org/jira/browse/CLI-1
7,Cli,7,Bug27575Test,testRequiredOptions,4,[Timestamp('2004-04-22 19:00:16')],['8f95e4a724350f9f80429c2af1c3ac9bb2b2c2db'],[True],2006-07-09 00:05:42,2008-02-23 20:08:31,False,False,https://issues.apache.org/jira/browse/CLI-121
12,Cli,11,HelpFormatterTest,testPrintOptionWithEmptyArgNameUsage,4,[Timestamp('2008-05-30 06:22:49')],['d36adebd3547279b709960c902c3fb7b89a9a4ef'],[True],2002-10-24 22:32:22,2007-03-15 16:49:08,False,False,https://issues.apache.org/jira/browse/CLI-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,Mockito,37,SpyingOnInterfacesTest,shouldFailFastWhenCallingRealMethodOnInterface,4,[Timestamp('2009-11-08 14:32:25')],['c0222c2dbfbd2b053961a46e4c2a3973aec55a75'],[True],2014-12-15 23:25:09,2014-12-15 23:25:57,False,False,https://github.com/mockito/mockito/issues/140
1579,Mockito,38,ActualInvocationHasNullArgumentNPEBugTest,shouldAllowPassingNullArgument,4,[Timestamp('2009-07-09 07:23:58')],['c1f2c4e6b5bab4cfc004f22999e11e8ac140d377'],[True],2014-09-09 15:13:44,2016-07-03 20:28:43,False,False,https://github.com/mockito/mockito/issues/79
1580,Mockito,38,ArgumentMatchingToolTest,shouldWorkFineWhenGivenArgIsNull,4,[Timestamp('2009-07-09 07:23:58')],['c1f2c4e6b5bab4cfc004f22999e11e8ac140d377'],[True],2014-09-09 15:13:44,2016-07-03 20:28:43,False,False,https://github.com/mockito/mockito/issues/79
1581,Time,1,TestPartial_Constructors,testConstructorEx7_TypeArray_intArray,4,[Timestamp('2005-06-26 19:29:36')],['b86efe73e72ece8e5ceeed48d05228aa1725c9ad'],[True],2013-12-01 09:33:58,2013-12-02 19:14:54,False,False,https://github.com/JodaOrg/joda-time/issues/93


### based on the report resolution, find the nearest commit


In [6]:
%run "gzoltar_utils.ipynb"

import shutil
from lxml import etree, objectify

pom_file = "/Users/IceCream/Documents/defects4j_projects/test_clone/gson/gson/pom.xml"
add_gzoltar(pom_file)

/var/folders/mx/yyskt9h51ls4lb4pqjfjc8jr0000gn/T/ipykernel_60555/3777084551.py:34: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not plugins:


### Gzoltar

In [ ]:
%run "gzoltar_utils.ipynb"

import time
import shutil
from lxml import etree, objectify

commits = ['']

for commit in commits:
    print(commit)
    
    destination = os.path.join(out_dir, "spectrum", proj, commit)
#     if os.path.exists(destination):
#         continue
    
    # checkout
    execute_shell_file("../sh/git_checkout.sh", [proj_dir, commit])

    # adding gzoltar
    print("adding gzoltar into root pom.xml")
    add_gzoltar(pom_file)
    
    start = time.time()
    
    # run mvn commands
    print("running mvn commands")
    extra_command = ""
    if proj == "gson":
        extra_command = "-Dadditionalparam=-Xdoclint:none"
    execute_shell_file_poll('../sh/mvn_gzoltar.sh', [proj_dir, extra_command])
    
    end = time.time()
    print('time elapsed', end - start)
        
    # store out
    shutil.copytree(sfl_out_dir, destination, dirs_exist_ok=True)
    
    break


In [ ]:
%run "gzoltar_utils.ipynb"

import re
import numpy as np

formulas = ['tarantula', 'ochiai', 'dstar', 'barinel']

test_failure = json_to_dict(os.path.join(out_dir, "empirical_cov4change", proj, "coverage_diff.json"))
introducing_list = list(set([introducing_commit
    for t, commit_dict in test_failure.items() 
    for introducing_commit in commit_dict
]))

visited = []
for commit in introducing_list:
    if commit in visited:
        print(commit, "visited")
        continue
    commit_dir = os.path.join(out_dir, "spectrum", proj, commit)
    
    for a_f in formulas:
        processed_file = os.path.join(commit_dir, "processed", a_f+'.csv')
        # skip if processed
        if os.path.exists(processed_file):
            continue
            
        df_rank_processed = []
        rank_file = os.path.join(commit_dir, a_f+'.ranking.csv')
        print(rank_file)
        if not os.path.exists(rank_file):
            continue
            
        df_rank = pd.read_csv(rank_file, sep=';')
        # remove suspiciousness_value == 0, save time for objectify
        df_rank['suspiciousness_value'] = df_rank['suspiciousness_value'].apply(lambda x: float(x))
        df_rank = df_rank[df_rank['suspiciousness_value'] != 0]
        df_rank = df_rank.apply(objectify_name, axis=1)
        df_rank = df_rank.drop_duplicates(subset=['package', 'class', 'method'], keep='first')
        df_rank.index = np.arange(1, len(df_rank) + 1)
        display(df_rank)
        
        processed_dir = os.path.join(commit_dir, "processed")
        if not os.path.exists(processed_dir):
            os.makedirs(processed_dir, exist_ok=True)
        df_rank.to_csv(os.path.join(processed_dir, a_f+'.csv'))
        
    visited.append(commit)

In [ ]:

dst_path = os.path.join(lorena_out, proj, commit)

print(proj, commit)

#test_list_file = os.path.join(out_dir, "builds", proj, commit+".json")
test_list_file = os.path.join(dst_path, "surefire_test_method_dict.json")
if not os.path.exists(test_list_file):
    print("Test dict not available", test_list_file)
    continue
test_dict = json_to_dict(test_list_file)
# remove test methods
test_dict = {t:[] for t in test_dict[commit]}
original_test = None
driver(proj, commit, test_dict, original_test)